In [52]:
from PIL import Image
import cv2
import pytesseract
import numpy as np
from fuzzywuzzy import fuzz
from typing import OrderedDict
import re
import fitz
import os
import glob
import time
import img2pdf
import shutil
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import matplotlib.pyplot as plt

In [96]:
in_path = '/home/vimalkumar/Documents/redact_project/pdf/ceremony/pdf'
out_path = '/home/vimalkumar/Documents/redact_project/pdf/ceremony/images'

for pdf_file in os.listdir(in_path):
    print(pdf_file)
    
    pdf_extension = os.path.splitext(pdf_file)[1].lower()
    pdf_name = os.path.splitext(pdf_file)[0]
    os.makedirs(os.path.join(out_path,pdf_name))
    
    #print(pdf_extension)
    if pdf_extension.endswith('.pdf'):
        doc = fitz.open(os.path.join(in_path,pdf_file))
        print(len(doc))
        for i in range(len(doc)):
            doc = fitz.open(os.path.join(in_path,pdf_file))
            #print(len(doc))
            page = doc.load_page(i)  # number of page
            pix = page.get_pixmap(matrix=fitz.Matrix(150/72,150/72))# for 150dpi
            file_name = str(i+1000 + 1)       
            pix.pil_save(os.path.join(out_path,pdf_name,file_name+'.jpeg'))
     
        data_dict_dob=OrderedDict({
            "No:":{"label":"No:",'Zoning_type':'inline'},
            "Account":{"label":"Account",'Zoning_type':'inline'}
        })

        files = ['1010.jpeg','1012.jpeg']
        
        #for file in files:
        for file in files:
            image=cv2.imread(os.path.join(out_path,pdf_name,file),0)
            #image_w=cv2.imread(os.path.join(out_path,pdf_name,file))
            image_data=pytesseract.image_to_data(image,config="--psm 6",output_type=pytesseract.Output.DICT)
            #print(image_data["text"])
            #plt.imshow(image)

            tem_string_inline_net=""
            tem_string_bottom_net=""
            result_inline=[]
            for key in data_dict_dob.keys():
                to_find=data_dict_dob[key]["label"].lower()
                for i in range(len(image_data["text"])):
                    if to_find in image_data["text"][i].lower():
                        x,y,w,h=image_data["left"][i],image_data["top"][i],image_data["width"][i],image_data["height"][i]
                        if data_dict_dob[key]["label"] == "Account":
            #                     print("in")
                            zoning_area=(x-20,y-10,w+x+500,h+y+40) 
                            #cv2.rectangle(image,(zoning_area[0],zoning_area[1]),(zoning_area[2],zoning_area[3]),(0,0,255),4)
                            #plt.figure(figsize=(10,10))
                            #plt.imshow(image)
                        else:
                            zoning_area=(x-20,y-10,w+x+500,h+y+40) 
                            #cv2.rectangle(image,(zoning_area[0],zoning_area[1]),(zoning_area[2],zoning_area[3]),(0,0,255),4)
            #                                
                        for j in range(len(image_data["text"])):
                            x,y,w,h=image_data["left"][j],image_data["top"][j],image_data["width"][j],image_data["height"][j]
                            if x>=zoning_area[0] and y>=zoning_area[1] and x+w <= zoning_area[2] and y+h <= zoning_area[3]:
                                tem_string_inline_net += image_data["text"][j]+" "

            #print(tem_string_inline_net)
            output_inline=re.findall(r"\d+",tem_string_inline_net)
            #     print(output_inline)


            if output_inline == []:
                print("nothing to mask")
            else:
                string_final=""
                for val in output_inline[0]:
                    string_final += val
                print(string_final)

                data_dict_fin=OrderedDict({
                        "Account":{"label":string_final}
                        })    


                for key in data_dict_fin.keys():
                    to_find=data_dict_fin[key]["label"].lower()
                    for fin in range(len(image_data["text"])):
                        if to_find in image_data["text"][fin].lower():
                            x,y,w,h=image_data["left"][fin],image_data["top"][fin],image_data["width"][fin],image_data["height"][fin]
                            if data_dict_fin[key]["label"] == string_final and len(string_final) == 9:
                                zoning_area=(x,y,w+x-39,h+y)
                                cv2.rectangle(image,(zoning_area[0],zoning_area[1]),(zoning_area[2],zoning_area[3]),(0,0,0),-1)
    #                             plt.figure(figsize=(15,15))
    #                             plt.imshow(image)
                                final_image =  Image.fromarray(image)
                                final_image.save(os.path.join(out_path,pdf_name,file))
                            if data_dict_fin[key]["label"] == string_final and len(string_final) != 9:
                                zoning_area=(x,y,w+x-35,h+y)
                                cv2.rectangle(image,(zoning_area[0],zoning_area[1]),(zoning_area[2],zoning_area[3]),(0,0,0),-1)
    #                             plt.figure(figsize=(15,15))
    #                             plt.imshow(image)
                                final_image =  Image.fromarray(image)
                                final_image.save(os.path.join(out_path,pdf_name,file))
            
        image=cv2.imread(os.path.join(out_path,pdf_name,'1011.jpeg'))
        gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        ret,thresh=cv2.threshold(gray,127,255,cv2.THRESH_BINARY)
        inv= 255-thresh
        length = np.array(inv).shape[1]//100
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (length, 1))
        horizontal_detect = cv2.erode(inv, horizontal_kernel,iterations=3)
        hor_lines = cv2.dilate(horizontal_detect, horizontal_kernel)
        horizontal_inv= 255-hor_lines
        final = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
        combine_1 = cv2.erode(horizontal_inv, final, iterations=5)
        thresh, combined = cv2.threshold(combine_1,128,255, cv2.THRESH_BINARY)
        img_3d=cv2.merge([combined,combined,combined])
        contours,_= cv2.findContours(combined,cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        imageCopy=img_3d.copy
        if len(contours)==13:
            selected_contours=[]
            for cnt in contours:
                area = cv2.contourArea(cnt)
                # print(area)
                if area < 2100826.0:
                    selected_contours.append(cnt)



            img= image.copy()
            cont1 = selected_contours[11]
            cont2= selected_contours[4]



            x,y,w,h = cv2.boundingRect(cont1)
            x1,y1,w1,h1=cv2.boundingRect(cont2)

            cv2.rectangle(img,(x1+w1+15,y1+10),(x-230,y+h-120),(0,0,0),-3)
            crop_img = Image.fromarray(img)
            crop_img.save(os.path.join(out_path,pdf_name,'1011.jpeg'))
            # cropped=img[y1:y+h-100,x1+w1:350,::-1]
#             plt.figure(figsize=(15,15))
#             plt.imshow(img,cmap="gray")
#             plt.show()



        else:
            print('nothing to mask')
#             plt.figure(figsize=(15,15))
#             plt.imshow(image)

        
for folder in os.listdir(out_path):      
    print(folder)
    with open(os.path.join(out_path,folder+".pdf"),"wb") as f:
        f.write(img2pdf.convert(sorted(glob.glob(os.path.join(out_path,folder,'*.jpeg')))))
        
        



signed_1.pdf
12
197389566
nothing to mask
nothing to mask
signed.pdf
14
nothing to mask
0320611001563
nothing to mask
Ceremony.pdf
13
31062918
nothing to mask
Ceremony
signed_1
signed


In [93]:
import pytesseract
from pdf2image import convert_from_path
import PyPDF2
import io

In [95]:
pdf_writer = PyPDF2.PdfFileWriter()
path = '/home/vimalkumar/Documents/redact_project/pdf/ceremony/images/Ceremony'
for image in os.listdir(path):
    page = pytesseract.image_to_pdf_or_hocr(os.path.join(path,image), extension='pdf')
    pdf = PyPDF2.PdfFileReader(io.BytesIO(page))
    pdf_writer.addPage(pdf.getPage(0))
# export the searchable PDF to searchable.pdf
with open("searchable.pdf", "wb") as f:
    pdf_writer.write(f)

In [ ]:
all_files = []
for (path,dirs,files) in os.walk('/home/vimalkumar/Documents/redact_project/pdf/ceremony/images'):
    for file in files:
        file = os.path.join(path, file)
        all_files.append(file)
pdf_writer = PyPDF2.PdfFileWriter()
for file in all_files:
    page = pytesseract.image_to_pdf_or_hocr(file, extension='pdf')
    pdf = PyPDF2.PdfFileReader(io.BytesIO(page))
    pdf_writer.addPage(pdf.getPage(0))
    
with open("searchable.pdf", "wb") as f:
    pdf_writer.write(f)